In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV fil
market_df_scaled = StandardScaler().fit_transform(df_market_data[["price_change_percentage_24h" , "price_change_percentage_7d", "price_change_percentage_14d", "price_change_percentage_30d", "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"]]
                                                 )

In [ ]:
# Create a DataFrame with the scaled data
market_df_scaled = pd.DataFrame(
    market_df_scaled,
    columns=["price_change_percentage_24h" , "price_change_percentage_7d", "price_change_percentage_14d", "price_change_percentage_30d", "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"])

# Copy the crypto names from the original data
market_df_scaled["coin_id"] = df_market_data.index
# Set the coinid column as index
market_df_scaled = market_df_scaled.set_index("coin_id")

# Display sample data
market_df_scaled.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
inertia = []
k = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values
for i in k:
    k_model = KMeans(n_clusters=i, random_state=0)
    k_model.fit(market_df_scaled)
    inertia.append(k_model.inertia_)

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 3

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(market_df_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_clusters_predictions = model.predict(market_df_scaled)


# Print the resulting array of cluster values.
print(crypto_clusters_predictions)

In [ ]:
# Create a copy of the DataFrame
df_predicitons = market_df_scaled.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
df_predicitons["CryptoCluster"] = crypto_clusters_predictions

# Display sample data
df_predicitons.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_predicitons.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="CryptoCluster",
    hover_cols = ["coin_id"], 
    title = "Scatter Plot by crypto coin id - k=4"
)

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
crypto_pca_data = pca.fit_transform(market_df_scaled)
# View the first five rows of the DataFrame. 
crypto_pca_data[:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_ 

In [ ]:
import numpy as np
total_explained_variance = np.sum(pca.explained_variance_ratio_)
total_explained_variance

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** ~90%

In [ ]:
# Create a new DataFrame with the PCA data.
df_crypto_pca = pd.DataFrame(crypto_pca_data, columns=["PC1", "PC2", "PC3"])
# Creating a DataFrame with the PCA data

# Copy the crypto names from the original data
df_crypto_pca["coin_id"] = df_market_data.index

# Set the coinid column as index
df_crypto_pca = df_crypto_pca.set_index("coin_id")

# Display sample data
df_crypto_pca.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_pca = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k_pca:
    k_model = KMeans(n_clusters=i, random_state=0)
    k_model.fit(df_crypto_pca)
    inertia.append(model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
    "k_pca": k_pca,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)
df_elbow_pca.head()

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow_pca.hvplot.line(x="k_pca", y="inertia", title="Elbow Curve Using PCA Data", xticks=k)
elbow_plot_pca

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 3


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** The best k value in the orginial data was 4 and now based on the elbow plot the best value is 3

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=3)

In [ ]:
# Fit the K-Means model using the PCA data
model.fit(df_crypto_pca)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
crypto_clusters = model.predict(df_crypto_pca)
# Print the resulting array of cluster values.
print(crypto_clusters)

In [ ]:
# Create a copy of the DataFrame with the PCA data
df_crypto_pca_predictions = df_crypto_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_crypto_pca_predictions["pca_predictions"] = crypto_clusters

# Display sample data
df_crypto_pca_predictions.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_crypto_pca_predictions.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="pca_predictions",
    hover_cols=['coin_id'],
    title = "Scatter Plot by Stock Segment - PCA=2"
)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
df_elbow + elbow_plot_pca

In [ ]:
# Composite plot to contrast the clusters
df_predicitons + df_crypto_pca_predictions

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 